In [4]:
using GATlab


In [5]:
@symbolic_model CategoryExprs{ObExpr, HomExpr} ThCategory begin
end
A, B, C, D = [ Ob(CategoryExprs.Ob, X) for X in [:A, :B, :C, :D] ]
f, g, h = Hom(:f, A, B), Hom(:g, B, C), Hom(:h, C, D)
compose(compose(f,g),h)

UndefVarError: UndefVarError: `ObExpr` not defined

In [6]:
a = ThCategory.Hom[FinSetC()]([1,2,2]::Vector{Int}, 3::Int, 2::Int)

3-element Vector{Int64}:
 1
 2
 2

In [7]:
b = ThCategory.Ob[FinSetC()]()

MethodError: MethodError: no method matching Ob(::GATlab.Syntax.TheoryInterface.WithModel{FinSetC}; context::Nothing)

Closest candidates are:
  Ob(::GATlab.Syntax.TheoryInterface.WithModel{FinSetC}, !Matched::Int64; context)
   @ GATlab none:0
  Ob(!Matched::GATlab.Syntax.TheoryInterface.WithModel{OpC{ObT, HomT, C}}, !Matched::ObT; context) where {ObT, HomT, C}
   @ GATlab none:0
  Ob(!Matched::GATlab.Syntax.TheoryInterface.WithModel{SliceC{ObT, HomT, C}}, !Matched::SliceOb{ObT, HomT}; context) where {ObT, HomT, C}
   @ GATlab none:0
  ...


In [8]:
ThCategory.Ob[FinSetC()](1)

1

In [9]:
@withmodel FinSetC() (Ob, Hom, id, compose, dom, codom) begin
    
c= Ob(1)
d= Ob(2)
Hom([2], c::Int, d::Int)

end



UndefVarError: UndefVarError: `Ob` not defined

In [10]:
getscope(ThCategory)

MethodError: MethodError: no method matching getscope(::Module)

Closest candidates are:
  getscope(!Matched::Scope)
   @ GATlab C:\Users\Paulius\.julia\packages\GATlab\M0FGy\src\syntax\Scopes.jl:414
  getscope(!Matched::HasScopeList, !Matched::Int64)
   @ GATlab C:\Users\Paulius\.julia\packages\GATlab\M0FGy\src\syntax\Scopes.jl:749
  getscope(!Matched::AppendContext, !Matched::Int64)
   @ GATlab C:\Users\Paulius\.julia\packages\GATlab\M0FGy\src\syntax\Scopes.jl:784
  ...


In [11]:
@withmodel FinSetC() (Ob, Hom, id, compose, dom, codom) begin
    
c= Ob(1)
d= Ob(2)
Hom([2], c::Int, d::Int)

end

UndefVarError: UndefVarError: `Ob` not defined

In [12]:
struct TestCase{T, O}
    input::T
    expectedOutput::O
end

const FirstTestCase = TestCase{Tuple{Int, Int, Int}, Int}

example1 = FirstTestCase((1,2,3), 3)

FirstTestCase((1, 2, 3), 3)

In [13]:
struct FirstTestSuiteC <: Model{Tuple{FirstTestCase, Tuple{FirstTestCase, FirstTestCase}}} end

function is_permutation(tuple_l::Tuple, tuple_r::Tuple) :: Bool 
    return sort(collect(tuple_l)) == sort(collect(tuple_r))
end

#Max out of 3
@instance GATlab.ThThinCategory{FirstTestCase, Tuple{FirstTestCase, FirstTestCase}} [model::FirstTestSuiteC] begin

    Ob(x::FirstTestCase) = maximum(x.input) == x.expectedOutput ? x : @fail "Invalid test case."

    function Hom(f::Tuple{FirstTestCase, FirstTestCase}, dom::FirstTestCase, codom::FirstTestCase)
        if is_permutation(dom.input, codom.input)
            return f
        else
            @fail "Test case input arrays are not permutations of each other."
        end
    end

    id(testcase::FirstTestCase) = Tuple{FirstTestCase, FirstTestCase}((testcase, testcase))

    compose(f::Tuple{FirstTestCase, FirstTestCase}, g::Tuple{FirstTestCase, FirstTestCase}) = 
                            f[2] == g[1] ? Tuple{FirstTestCase, FirstTestCase}((f[1], g[2])) : @fail "Domain and codomain of morphisms must match in order to perform composition"
    
    dom(f::Tuple{FirstTestCase, FirstTestCase}) = f[1]
    codom(f::Tuple{FirstTestCase, FirstTestCase}) = f[2]

end

##docsink#324 (generic function with 0 methods)

In [14]:
c = GATlab.ThThinCategory.Ob[FirstTestSuiteC()](FirstTestCase((1,2,3),3))
d = GATlab.ThThinCategory.Ob[FirstTestSuiteC()](FirstTestCase((1,3,2),3))
e = GATlab.ThThinCategory.Ob[FirstTestSuiteC()](FirstTestCase((3,1,2),3))

f = GATlab.ThThinCategory.Hom[FirstTestSuiteC()]((c,d)::Tuple{FirstTestCase, FirstTestCase}, c, d)
g = GATlab.ThThinCategory.Hom[FirstTestSuiteC()]((d,e)::Tuple{FirstTestCase, FirstTestCase}, d, e)

h = GATlab.ThThinCategory.compose[FirstTestSuiteC()](f,g)

somethjing = GATlab.ThThinCategory.Hom[FirstTestSuiteC()]((c,d)::Tuple{FirstTestCase, FirstTestCase}, c, d)

somethjing == f


# c_id = GATlab.ThThinCategory.id[FirstTestSuiteC()](c)
# GATlab.ThThinCategory.compose[FirstTestSuiteC()](c_id,f)

true

In [15]:
const SecondTestCase = TestCase{Int, Int}

example2 = SecondTestCase(1,0)

struct SecondTestSuiteC <: Model{Tuple{SecondTestCase, Tuple{Int, Int}}} end

# Is a given number even
@instance GATlab.ThThinCategory{SecondTestCase, Tuple{Int, Int}} [model::SecondTestSuiteC] begin
    
    Ob(x::SecondTestCase) = abs(x.input % 2 - 1) == x.expectedOutput ? x : @fail "Invalid test case."

    Hom(f::Tuple{Int, Int}, dom::SecondTestCase, codom::SecondTestCase) = f[2] % 2 == 0 && dom.input + f[2] == codom.input && f[1] == dom.input ? f : @fail "Invalid morphism"

    id(testcase::SecondTestCase) = Tuple{Int, Int}(testcase.input, 0)

    compose(f::Tuple{Int,Int}, g::Tuple{Int,Int}) = f[1] + f[2] == g[1] ? (f[1], g[1] + g[2] - f[1])::Tuple{Int,Int} : @fail "Invalid composition"

    dom(f::Tuple{Int, Int}) = f[1]

    codom(f::Tuple{Int, Int}) = f[1] + f[2]

end

# Test modelio kategorija produktas su sample kategorija
# Random generating objects from category
# Sampling kategorija kaip urna
# Objektas - test case ištraukimo operacija
# Morfizmas - sugalvoti kaip realizuoti traukimą sekančio objekto
# Urna virs baigtines aibes. Jeigu traukimas vyksta už baigtinės aibės ribų - traukimo rezultatas ::Nothing
# Baigtine tikimybine erdve

##docsink#337 (generic function with 0 methods)

In [16]:
a = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(1,0))
b = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(2,1))
c = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(3,0))
d = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(5,0))


f = GATlab.ThThinCategory.Hom[SecondTestSuiteC()]((a.input,2)::Tuple{Int, Int}, a, c)
g = GATlab.ThThinCategory.Hom[SecondTestSuiteC()]((c.input,2)::Tuple{Int, Int}, c, d)

h = GATlab.ThThinCategory.compose[SecondTestSuiteC()](f,g)

(1, 4)

In [17]:
const ThirdTestCase = TestCase{Vector{Int}, Int}

example3 = ThirdTestCase([1,2,3,4], 10)

struct ThirdTestSuiteC <: Model{Tuple{ThirdTestCase, Tuple{ThirdTestCase, ThirdTestCase}}} end

# Sum of vector of integers
@instance GATlab.ThThinCategory{ThirdTestCase, Tuple{ThirdTestCase, ThirdTestCase}} [model::ThirdTestSuiteC] begin
    Ob(x::ThirdTestCase) = sum(x.input) == x.expectedOutput ? x : @fail "Invalid object"

    function Hom(f::Tuple{ThirdTestCase, ThirdTestCase}, dom::ThirdTestCase, codom::ThirdTestCase) 
        A = f[2].input
        B = f[1].input
        
        count_A = Dict()

        for element in A
            count_A[element] = get(count_A, element, 0) + 1
        end

        for element in B
            if !haskey(count_A, element) || count_A[element] == 0
                @fail "Invalid morphism"
            else
                count_A[element] -= 1
            end
        end
    
        return f
    end

    id(x::ThirdTestCase) = (x, x)

    compose(f::Tuple{ThirdTestCase, ThirdTestCase}, g::Tuple{ThirdTestCase, ThirdTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{ThirdTestCase, ThirdTestCase}) = f[1]

    codom(f::Tuple{ThirdTestCase, ThirdTestCase}) = f[2]
end

##docsink#350 (generic function with 0 methods)

In [18]:

a = GATlab.ThThinCategory.Ob[ThirdTestSuiteC()](ThirdTestCase([1],1))
b = GATlab.ThThinCategory.Ob[ThirdTestSuiteC()](ThirdTestCase([1,3],4))
c = GATlab.ThThinCategory.Ob[ThirdTestSuiteC()](ThirdTestCase([1,3,6],10))

f = GATlab.ThThinCategory.Hom[ThirdTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[ThirdTestSuiteC()]((b,c),b,c) 
id = GATlab.ThThinCategory.id[ThirdTestSuiteC()](b)

composition = GATlab.ThThinCategory.compose[ThirdTestSuiteC()](f,id)

dom = GATlab.ThThinCategory.dom[ThirdTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[ThirdTestSuiteC()](composition)

show(dom)
show(codom)

ThirdTestCase([1], 1)ThirdTestCase([1, 3], 4)

In [19]:
const FourthTestCase = TestCase{Vector{Int}, Vector{Int}}

example4 = FourthTestCase([1,3,2],[2,3,1])

struct FourthTestSuiteC <: Model{Tuple{FourthTestCase, Tuple{FourthTestCase, FourthTestCase}}} end

# Finite list reverse
@instance GATlab.ThThinCategory{FourthTestCase, Tuple{FourthTestCase, FourthTestCase}} [model::FourthTestSuiteC] begin
    Ob(x::FourthTestCase) = x.input == reverse(x.expectedOutput) ? x : @fail "Invalid object"

    function Hom(f::Tuple{FourthTestCase, FourthTestCase}, dom::FourthTestCase, codom::FourthTestCase)
        dom_input = f[1].input
        codom_input = f[2].input

        dom_expectedoutput = f[1].expectedOutput
        codom_expectedoutput = f[2].expectedOutput

        if length(dom_input) > length(codom_input) || any(dom_input[i] != codom_input[i] for i in 1:length(dom_input))
            @fail "Domain input array is not a subsequence of codomain input array"
        end
    
        if length(dom_expectedoutput) > length(codom_expectedoutput)
           @fail "domain expected output array is not a subsequence of codomain expected output array" 
        end
        
        for i in 1:length(dom_expectedoutput) 
            if dom_expectedoutput[i] != codom_expectedoutput[i+length(codom_expectedoutput)-length(dom_expectedoutput)]
                @fail "domain expected output array is not a subarray of codomain expected output array"
            end
        end

        return f
    end

    id(x::FourthTestCase) = (x, x)

    compose(f::Tuple{FourthTestCase, FourthTestCase}, g::Tuple{FourthTestCase, FourthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{FourthTestCase, FourthTestCase}) = f[1]

    codom(f::Tuple{FourthTestCase, FourthTestCase}) = f[2]
end

##docsink#363 (generic function with 0 methods)

In [20]:
a = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,2,3],[3,2,1]))
b = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,3,2],[2,3,1]))
c = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,3,2,4],[4,2,3,1]))
d = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,3,2,4,8],[8,4,2,3,1]))

f = GATlab.ThThinCategory.Hom[FourthTestSuiteC()]((b,c), b, c)
g = GATlab.ThThinCategory.Hom[FourthTestSuiteC()]((c,d), c, d)

h = GATlab.ThThinCategory.Hom[FourthTestSuiteC()]((b,d), b, d)
id = GATlab.ThThinCategory.id[FourthTestSuiteC()](a)

compose = GATlab.ThThinCategory.compose[FourthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[FourthTestSuiteC()](compose)
codom = GATlab.ThThinCategory.codom[FourthTestSuiteC()](compose)

show(dom)
show(codom)



FourthTestCase([1, 3, 2], [2, 3, 1])FourthTestCase(

[1, 3, 2, 4, 8], [8, 4, 2, 3, 1])

In [21]:
const FifthTestCase = TestCase{Vector{Int}, Vector{Int}}

example5 = FifthTestCase([1,3,2],[1,2,3])

struct FifthTestSuiteC <: Model{Tuple{FifthTestCase, Tuple{FifthTestCase, FifthTestCase}}} end

# Sorting a finite list in ascending order
@instance GATlab.ThThinCategory{FifthTestCase, Tuple{FifthTestCase, FifthTestCase}} [model::FifthTestSuiteC] begin
    Ob(x::FifthTestCase) = sort(x.input) == x.expectedOutput ? x : @fail "Invalid object"
    
    function Hom(f::Tuple{FifthTestCase, FifthTestCase}, dom::FifthTestCase, codom::FifthTestCase)
        A = f[2].input
        B = f[1].input
        
        count_A = Dict()

        for element in A
            count_A[element] = get(count_A, element, 0) + 1
        end

        for element in B
            if !haskey(count_A, element) || count_A[element] == 0
                @fail "Invalid morphism"
            else
                count_A[element] -= 1
            end
        end
    
        return f
    end

    id(x::FifthTestCase) = (x, x)

    compose(f::Tuple{FifthTestCase, FifthTestCase}, g::Tuple{FifthTestCase, FifthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{FifthTestCase, FifthTestCase}) = f[1]

    codom(f::Tuple{FifthTestCase, FifthTestCase}) = f[2]
end

##docsink#376 (generic function with 0 methods)

In [22]:
a = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([1,3,2],[1,2,3]))
b = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([3,1,2],[1,2,3]))
c = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([1,3,2,4],[1,2,3,4]))
d = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([1,2,2,4],[1,2,2,4]))

f = GATlab.ThThinCategory.Hom[FifthTestSuiteC()]((a,b), a, b)
g = GATlab.ThThinCategory.Hom[FifthTestSuiteC()]((b,c), b, c)

id = GATlab.ThThinCategory.id[FifthTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[FifthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[FifthTestSuiteC()](composition)

codom = GATlab.ThThinCategory.dom[FifthTestSuiteC()](composition)

show(dom)
show(codom)

TestCase{Vector

{Int64}, Vector{Int64}}([1, 3, 2], [1, 2, 3])TestCase{Vector{Int64}, Vector{Int64}}([1, 3, 2], [1, 2, 3])

In [23]:
const SixthTestCase = TestCase{Vector{Vector{Int}}, Vector{Int}}

example6 = SixthTestCase([[1,2],[3,4]],[1,2,3,4])

struct SixthTestSuiteC <: Model{Tuple{SixthTestCase, Tuple{SixthTestCase, SixthTestCase}}} end

# Finite list concatenation
@instance GATlab.ThThinCategory{SixthTestCase, Tuple{SixthTestCase, SixthTestCase}} [model::SixthTestSuiteC] begin
    Ob(x::SixthTestCase) = collect(Base.Iterators.Flatten(x.input)) == x.expectedOutput ? x : @fail "Invalid object"

    function Hom(f::Tuple{SixthTestCase, SixthTestCase}, dom::SixthTestCase, codom::SixthTestCase) 
        dom_flatinput = collect(Base.Iterators.Flatten(f[1].input))
        codom_flatinput = collect(Base.Iterators.Flatten(f[2].input))

        if length(dom_flatinput) > length(codom_flatinput) || any(dom_flatinput[i] != codom_flatinput[i] for i in 1:length(dom_flatinput))
            @fail "Domain input array is not a subsequence of codomain input array"
        end

        return f
    end
    
    id(x::SixthTestCase) = (x, x)

    compose(f::Tuple{SixthTestCase, SixthTestCase}, g::Tuple{SixthTestCase, SixthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{SixthTestCase, SixthTestCase}) = f[1]

    codom(f::Tuple{SixthTestCase, SixthTestCase}) = f[2]
end

##docsink#389 (generic function with 0 methods)

In [24]:
a = GATlab.ThThinCategory.Ob[SixthTestSuiteC()](SixthTestCase([[1,3,2]],[1,3,2]))
b = GATlab.ThThinCategory.Ob[SixthTestSuiteC()](SixthTestCase([[1,3],[2]],[1,3,2]))
c = GATlab.ThThinCategory.Ob[SixthTestSuiteC()](SixthTestCase([[1,3],[2,2]],[1,3,2,2]))

f = GATlab.ThThinCategory.Hom[SixthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[SixthTestSuiteC()]((b,c),b,c)

id = GATlab.ThThinCategory.id[SixthTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[SixthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[SixthTestSuiteC()](composition)

codom = GATlab.ThThinCategory.codom[SixthTestSuiteC()](composition)

show(dom)
show(codom)


SixthTestCase([[1, 3, 2]], [1, 3, 2])SixthTestCase([[1, 3], [2, 2]], [1, 3, 2, 2])

In [25]:
const SeventhTestCase = TestCase{Tuple{Vector{Int}, Int}, Int}

example7 = SeventhTestCase(([1,2,2,3],2),2)

struct SeventhTestSuiteC <: Model{Tuple{SeventhTestCase, Tuple{SeventhTestCase, SeventhTestCase}}} end

# Count number of occurances of an element in a finite list
@instance GATlab.ThThinCategory{SeventhTestCase, Tuple{SeventhTestCase, SeventhTestCase}} [model::SeventhTestSuiteC] begin
    Ob(x::SeventhTestCase) = count(y -> y == x.input[2], x.input[1]) == x.expectedOutput ? x : @fail "Invalid object"

    function Hom(f::Tuple{SeventhTestCase, SeventhTestCase}, dom::SeventhTestCase, codom::SeventhTestCase)
        dom_inputvec = f[1].input[1]
        codom_inputvec = f[2].input[1]

        dom_countednumber = f[1].input[2]
        codom_countednumber = f[1].input[2]
        
        count_codom_inputvec = Dict()

        if dom_countednumber != codom_countednumber
            @fail "Counted number don't match"
        end

        for element in codom_inputvec
            count_codom_inputvec[element] = get(count_codom_inputvec, element, 0) + 1
        end

        for element in dom_inputvec
            if !haskey(count_codom_inputvec, element) || count_codom_inputvec[element] == 0
                @fail "Invalid morphism"
            else
                count_codom_inputvec[element] -= 1
            end
        end
    
        return f
    end

    compose(f::Tuple{SeventhTestCase, SeventhTestCase}, g::Tuple{SeventhTestCase, SeventhTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    id(x::SeventhTestCase) = (x,x)

    dom(f::Tuple{SeventhTestCase, SeventhTestCase}) = f[1]

    codom(f::Tuple{SeventhTestCase, SeventhTestCase}) = f[2]
end

##docsink#402 (generic function with 0 methods)

In [26]:
a = GATlab.ThThinCategory.Ob[SeventhTestSuiteC()](SeventhTestCase(([1,3,2], 2),1))
b = GATlab.ThThinCategory.Ob[SeventhTestSuiteC()](SeventhTestCase(([1,2,3], 2),1))
c = GATlab.ThThinCategory.Ob[SeventhTestSuiteC()](SeventhTestCase(([1,2,3,2], 2),2))

f = GATlab.ThThinCategory.Hom[SeventhTestSuiteC()]((a,b), a, b)
g = GATlab.ThThinCategory.Hom[SeventhTestSuiteC()]((b,c), b, c)

id = GATlab.ThThinCategory.id[SeventhTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[SeventhTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[SeventhTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[SeventhTestSuiteC()](composition)

show(dom)
show(codom)



SeventhTestCase(([1, 3, 2], 2), 1)SeventhTestCase(([1, 2, 3, 2], 2), 2)

In [27]:
const EighthTestCase = TestCase{Vector{Int}, Int} 

example8 = EighthTestCase(([1,2,2,3,4,4]),2)

struct EighthTestSuiteC <: Model{Tuple{EighthTestCase, Tuple{EighthTestCase, EighthTestCase}}} end

# Get most frequently repeating element in an array
@instance GATlab.ThThinCategory{EighthTestCase, Tuple{EighthTestCase, EighthTestCase}} [model::EighthTestSuiteC] begin
    function Ob(x::EighthTestCase)
        counts = Dict()
        input_vector = x.input
        expected_output = x.expectedOutput

        if isempty(input_vector)
            return x
        end

        for element in input_vector
            counts[element] = get(counts, element, 0) + 1
        end
        
        max_frequency = maximum(values(counts))

        most_frequent_elements = [element for (element, count) in counts if count == max_frequency]

        if in(expected_output, most_frequent_elements)
            return x
        end

        @fail "Invalid object."
    end

    function Hom(f::Tuple{EighthTestCase, EighthTestCase}, dom::EighthTestCase, codom::EighthTestCase)
        A = f[2].input
        B = f[1].input
        
        count_A = Dict()

        for element in A
            count_A[element] = get(count_A, element, 0) + 1
        end

        for element in B
            if !haskey(count_A, element) || count_A[element] == 0
                @fail "Invalid morphism"
            else
                count_A[element] -= 1
            end
        end
    
        return f
    end

    id(x::EighthTestCase) = (x, x)

    compose(f::Tuple{EighthTestCase, EighthTestCase}, g::Tuple{EighthTestCase, EighthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{EighthTestCase, EighthTestCase}) = f[1]

    codom(f::Tuple{EighthTestCase, EighthTestCase}) = f[2]
end

##docsink#415 (generic function with 0 methods)

In [28]:
a = GATlab.ThThinCategory.Ob[EighthTestSuiteC()](EighthTestCase([1],1))
b = GATlab.ThThinCategory.Ob[EighthTestSuiteC()](EighthTestCase([1,1],1))
c = GATlab.ThThinCategory.Ob[EighthTestSuiteC()](EighthTestCase([1,1,2],1))

f = GATlab.ThThinCategory.Hom[EighthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[EighthTestSuiteC()]((b,c),b,c)

composition = GATlab.ThThinCategory.compose[EighthTestSuiteC()](f,g)

id = GATlab.ThThinCategory.id[EighthTestSuiteC()](a)

dom = GATlab.ThThinCategory.dom[EighthTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[EighthTestSuiteC()](composition)

show(dom)
show(codom)

TestCase{Vector{Int64}, Int64}([1], 1)TestCase{Vector{Int64}, Int64}([1, 1, 2], 1)

In [29]:
const NinthTestCase = TestCase{Vector{Int}, Bool} # Check if a given array is a palindrome

example9 = NinthTestCase(([1,2,2,3,4,4]), false)

struct NinthTestSuiteC <: Model{Tuple{NinthTestCase, Tuple{NinthTestCase, NinthTestCase}}} end

# Check if a given vector is a palindrome
@instance GATlab.ThThinCategory{NinthTestCase, Tuple{NinthTestCase, NinthTestCase}} [model::NinthTestSuiteC] begin
    Ob(x::NinthTestCase) = x.expectedOutput == (reverse(x.input) == x.input) ? x : @fail "Invalid object"

    # Morphism can add same elements to both sides of vector if the vector is a palindrome.
    # It can also add element anywhere if the vector is not a palindrome
    function Hom(f::Tuple{NinthTestCase, NinthTestCase}, dom::NinthTestCase, codom::NinthTestCase)  
        dom_input = f[1].input
        codom_input = f[2].input

        if f[1].expectedOutput != f[2].expectedOutput
            @fail "Invalid morphism"
        end

        if dom_input == reverse(dom_input) && codom_input != reverse(codom_input)
            @fail "Invalid morphism"
        end

        if dom_input != reverse(dom_input) && codom_input == reverse(codom_input)
            @fail "Invalid morphism"
        end

        if f[1].expectedOutput == true
            for i in 1:(length(codom_input) - length(dom_input) + 1) 
                if codom_input[i:(i + length(dom_input) - 1)] == dom_input
                    return f
                end
            end
        else
            if any(dom_input[i] != codom_input[i] for i in 1:length(dom_input))
                @fail "Invalid morphism"
            end
            
            return f
        end
    end

    id(x::NinthTestCase) = (x, x)

    compose(f::Tuple{NinthTestCase, NinthTestCase}, g::Tuple{NinthTestCase, NinthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{NinthTestCase, NinthTestCase}) = f[1]

    codom(f::Tuple{NinthTestCase, NinthTestCase}) = f[2]
end

##docsink#428 (generic function with 0 methods)

In [30]:
a = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1], true))
b = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([2,1,2], true))
c = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([3,2,1,2,3], true))

test = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1,2,1], true))

f = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((b,c),b,c)

id = GATlab.ThThinCategory.id[NinthTestSuiteC()](a)

compose = GATlab.ThThinCategory.compose[NinthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[NinthTestSuiteC()](compose)
codom = GATlab.ThThinCategory.codom[NinthTestSuiteC()](compose)

a2 = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1, 2], false))
b2 = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1, 2, 3], false))
c2 = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1, 2, 3, 4], false))

f = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((a2,b2),a2,b2)
g = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((b2,c2),b2,c2)

e = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((b2,c2),b2,c2)

GATlab.ThThinCategory.compose[NinthTestSuiteC()](f,g)


(NinthTestCase([1, 2], false), NinthTestCase([1, 2, 3, 4], false))

In [52]:
const TenthTestCase = TestCase{Vector{Int}, Vector{Int}} # Check if a given array is a palindrome

example10 = TenthTestCase(([1,2,2,3,4,4]), [1,2,3,4])

struct TenthTestSuiteC <: Model{Tuple{TenthTestCase, Tuple{TenthTestCase, TenthTestCase}}} end

# Remove duplicate elements from a finite list
@instance GATlab.ThThinCategory{TenthTestCase, Tuple{TenthTestCase, TenthTestCase}} [model::TenthTestSuiteC] begin
    Ob(x::TenthTestCase) = Set(x.input) == Set(x.expectedOutput) ? x : @fail "Invalid object"

    function Hom(f::Tuple{TenthTestCase, TenthTestCase}, dom::TenthTestCase, codom::TenthTestCase)
        dom_input = f[1].input
        codom_input = f[2].input

        if issubset(Set(dom_input), Set(codom_input))
            return f
        end

        @fail "Invalid morphism"
    end

    id(x::TenthTestCase) = (x, x)

    compose(f::Tuple{TenthTestCase, TenthTestCase}, g::Tuple{TenthTestCase, TenthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{TenthTestCase, TenthTestCase}) = f[1]

    codom(f::Tuple{TenthTestCase, TenthTestCase}) = f[2]
end

##docsink#480 (generic function with 0 methods)

In [56]:
a = GATlab.ThThinCategory.Ob[TenthTestSuiteC()](TenthTestCase([1], [1]))
b = GATlab.ThThinCategory.Ob[TenthTestSuiteC()](TenthTestCase([1,1], [1]))
c = GATlab.ThThinCategory.Ob[TenthTestSuiteC()](TenthTestCase([1,1,2], [1,2]))

f = GATlab.ThThinCategory.Hom[TenthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[TenthTestSuiteC()]((b,c),b,c)

id = GATlab.ThThinCategory.id[TenthTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[TenthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[TenthTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[TenthTestSuiteC()](composition)

show(dom)
show(codom)

TestCase{Vector{Int64}, Vector{Int64}}([1], [1])TestCase{Vector{Int64}, Vector{Int64}}([1, 1, 2], [1, 2])

In [16]:
function get_random(vector::Vector{T}) where T
    if isempty(vector)
        return Nothing
    end

    index = rand(1:length(vector))
    return vector[index]
end

6

In [17]:
struct TestCaseSamplerC{T<:Any} <: Model{Tuple{Vector{T}, Tuple{Vector{T}, Vector{T}}}} end

@instance GATlab.ThThinCategory{Vector{T}, Tuple{Vector{T}, Vector{T}}} [model::TestCaseSamplerC{T}] where {T} begin
    Ob(vector::Vector{T}) = :(get_random($vector))

    Hom(f::Tuple{Vector{T}, Vector{T}}, dom::Vector{T}, codom::Vector{T}) = f

    id(vector::Vector{T}) = (vector, vector)

    compose(f::Tuple{Vector{T}, Vector{T}}, g::Tuple{Vector{T}, Vector{T}}) = Tuple(f[1], g[2])

    dom(f::Tuple{Vector{T}, Vector{T}}) = :(get_random($(f[1])))

    codom(f::Tuple{Vector{T}, Vector{T}}) = :(get_random($(f[2])))
end

##docsink#350 (generic function with 0 methods)

In [18]:
NumberLimit = 10

function initialize_testcases(limit::Int, condition::Function)
    testcases = [FirstTestCase((i, j, k), max(i, j, k)) for i in -50:limit, j in -50:limit, k in -50:limit if condition(i, j, k)]
    return reduce(vcat,testcases)
end

testcases1 = initialize_testcases(NumberLimit, (i,j,k) -> (i != j && i != k && j != k))
testcases2 = initialize_testcases(NumberLimit, (i,j,k) -> (count(x -> x == i, (i, j, k)) == 2 || count(x -> x == j, (i, j, k)) == 2 || count(x -> x == k, (i, j, k)) == 2))
testcases3 = initialize_testcases(NumberLimit, (i,j,k) -> (i == j == k))

61-element Vector{FirstTestCase}:
 FirstTestCase((-50, -50, -50), -50)
 FirstTestCase((-49, -49, -49), -49)
 FirstTestCase((-48, -48, -48), -48)
 FirstTestCase((-47, -47, -47), -47)
 FirstTestCase((-46, -46, -46), -46)
 FirstTestCase((-45, -45, -45), -45)
 FirstTestCase((-44, -44, -44), -44)
 FirstTestCase((-43, -43, -43), -43)
 FirstTestCase((-42, -42, -42), -42)
 FirstTestCase((-41, -41, -41), -41)
 ⋮
 FirstTestCase((2, 2, 2), 2)
 FirstTestCase((3, 3, 3), 3)
 FirstTestCase((4, 4, 4), 4)
 FirstTestCase((5, 5, 5), 5)
 FirstTestCase((6, 6, 6), 6)
 FirstTestCase((7, 7, 7), 7)
 FirstTestCase((8, 8, 8), 8)
 FirstTestCase((9, 9, 9), 9)
 FirstTestCase((10, 10, 10), 10)

In [19]:
function filter(vector::Vector{T}, criteria::Function) :: Vector{T} where {T}
    new_vector = [x for x in vector if criteria(x)]
    return length(new_vector) == 0 ? Vector{T}() : new_vector
end

filter_criteria = Function

struct TestCaseSamplerC2{T<:TestCase} <: Model{Tuple{Vector{T}, Tuple{Vector{T}, Function}}} end

@instance GATlab.ThCategory{Vector{T}, Tuple{Vector{T}, Function}} [model::TestCaseSamplerC2{T}] where {T} begin
    Ob(vector::Vector{T}) = :(get_random($vector))

    Hom(f::Tuple{Vector{T}, Function}, dom::Vector{T}, codom::Vector{T}) = filter(f[1], f[2]::Function) == codom ? (f[1],f[2]) : @fail "Invalid morphism"

    id(vector::Vector{T}) = (vector, ((x) -> true)::Function)

    compose(f::Tuple{Vector{T}, Function}, g::Tuple{Vector{T}, Function}) = filter(f[1], f[2]::Function) == g[1] ? (f[1], (x) -> f[2](x) && g[2](x) ) : @fail "Invalid composition"

    dom(f::Tuple{Vector{T}, Function}) = :(get_random($(f[1])))

    codom(f::Tuple{Vector{T}, Function}) = :(get_random($(filter(f[1],f[2]::Function))))
end

##docsink#363 (generic function with 0 methods)

In [43]:
function make_object(type::Type, object)
    GATlab.ThCategory.Ob[TestCaseSamplerC2{type}()](object)
end

function make_morphism(type::Type, hom, dom, codom)
    GATlab.ThCategory.Hom[TestCaseSamplerC2{type}()](hom, dom, codom)
end

make_object(FirstTestCase, all_same)
make_object(FirstTestCase, initial_testcases)

make_morphism(FirstTestCase, (initial_testcases,(x) -> x.input[1] == x.input[2] == x.input[3] ), initial_testcases, all_same)

MethodError: MethodError: no method matching Ob(::GATlab.Syntax.TheoryInterface.WithModel{TestCaseSamplerC2{FirstTestCase}}, ::Vector{SecondTestCase}; context::Nothing)

Closest candidates are:
  Ob(!Matched::GATlab.Syntax.TheoryInterface.WithModel{OpC{ObT, HomT, C}}, ::ObT; context) where {ObT, HomT, C}
   @ GATlab none:0
  Ob(!Matched::GATlab.Syntax.TheoryInterface.WithModel{TestCaseSamplerC{T}}, ::Vector{T}; context) where T
   @ Main none:0
  Ob(::GATlab.Syntax.TheoryInterface.WithModel{TestCaseSamplerC2{T}}, !Matched::Vector{T}; context) where T
   @ Main none:0


In [53]:
initial_testcases = initialize_testcases(10, (x,y,z) -> true)
all_same = filter(initial_testcases, (x) -> x.input[1] == x.input[2] == x.input[3])
all_same_positive = filter(all_same, (x) -> x.input[1] > 0)

two_same = filter(initial_testcases, (x) -> (x.input[1] == x.input[2] != x.input[3]) ||  (x.input[1] == x.input[3] != x.input[2]) || (x.input[2] == x.input[3] != x.input[1]))

all_different = filter(initial_testcases, (x) -> (x.input[1] != x.input[2]) && ( x.input[1] != x.input[3]) && (x.input[2] != x.input[3]))

initial = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](initial_testcases)

filtered_objects1 = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](all_same)
filtered_objects2 = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](all_same_positive)

filtered_objects3 = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](two_same)

f = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((initial_testcases, (x) -> x.input[1] == x.input[2] == x.input[3]), initial_testcases, all_same)
g = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((all_same, (x) -> x.input[1] > 0), all_same, all_same_positive)

h = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((initial_testcases, (x) -> (x.input[1] == x.input[2] != x.input[3]) ||  (x.input[1] == x.input[3] != x.input[2]) || (x.input[2] == x.input[3] != x.input[1])), initial_testcases, two_same)
id = GATlab.ThCategory.id[TestCaseSamplerC2{FirstTestCase}()](initial_testcases)

e = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((initial_testcases, (x) -> (x.input[1] != x.input[2]) && ( x.input[1] != x.input[3]) && (x.input[2] != x.input[3])), initial_testcases, all_different)




comp = GATlab.ThCategory.compose[TestCaseSamplerC2{FirstTestCase}()](f,g)

codom = GATlab.ThCategory.codom[TestCaseSamplerC2{FirstTestCase}()](e)

eval(filtered_objects1)

# Atvejis, kai galimi keli atsakymai


FirstTestCase((1, 1, 1), 1)

In [40]:
function initialize_testcases2(limit::Int)
    testcases = [SecondTestCase(i, i % 2) for i in -limit:limit]
    return reduce(vcat,testcases)
end

initial_testcases = initialize_testcases2(50)

even = filter(initial_testcases, (x) -> x.input % 2 == 0)
odd = filter(initial_testcases, (x) -> x.input % 2 == 1)

f = GATlab.ThCategory.Hom[TestCaseSamplerC2{SecondTestCase}()]((initial_testcases, (x) -> x.input % 2 == 0), initial_testcases, even)
g = GATlab.ThCategory.Hom[TestCaseSamplerC2{SecondTestCase}()]((initial_testcases, (x) -> x.input % 2 == 1), initial_testcases, odd)

codom = GATlab.ThCategory.codom[TestCaseSamplerC2{SecondTestCase}()](g)

eval(codom)

SecondTestCase(7, 1)

In [ ]:
function initialize_testcases2(limit::Int)
    testcases = [SecondTestCase(i, i % 2) for i in -limit:limit]
    return reduce(vcat,testcases)
end

initial_testcases = initialize_testcases2(50)

even = filter(initial_testcases, (x) -> x.input % 2 == 0)
odd = filter(initial_testcases, (x) -> x.input % 2 == 1)

f = GATlab.ThCategory.Hom[TestCaseSamplerC2{SecondTestCase}()]((initial_testcases, (x) -> x.input % 2 == 0), initial_testcases, even)
g = GATlab.ThCategory.Hom[TestCaseSamplerC2{SecondTestCase}()]((initial_testcases, (x) -> x.input % 2 == 1), initial_testcases, odd)

codom = GATlab.ThCategory.codom[TestCaseSamplerC2{SecondTestCase}()](g)

eval(codom)

In [ ]:
testcases = [1,2,3,4,5]

a = GATlab.ThThinCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](testcases1)
b = GATlab.ThThinCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](testcases2)
c = GATlab.ThThinCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](testcases3)

# f = GATlab.ThThinCategory.Hom[TestCaseSamplerC{Int}()]((testcases,testcases), testcases, testcases)

# id = GATlab.ThThinCategory.id[TestCaseSamplerC{Int}()](testcases)
# GATlab.ThThinCategory.Ob[TestCaseSamplerC{Int}()](GATlab.ThThinCategory.codom[TestCaseSamplerC{Int}()](f))
eval(a)

FirstTestCase((9, 7, 5), 9)

In [ ]:
const SampleSize = 50

object_set = 

GATlab.ThThinCategory.Ob[FirstTestSuiteC()]

In [309]:
@macroexpand (GATlab.ThThinCategory.Ob[TestCaseSamplerC{FirstTestCase}()](testcases1))

:((GATlab.ThThinCategory.Ob[TestCaseSamplerC{FirstTestCase}()])(testcases1))